# Kalman-Bucy Filter $\newcommand\pmatrix[1]{\begin{pmatrix}#1\end{pmatrix}}$

$\newcommand\L{\mathcal{L}}
\newcommand\P{\mathcal{P}}
\newcommand\K{\mathcal{K}}$

## Background

### Dynamic System

#### Determined
- Discrete: $$x_{j+1}=\Psi(x_j), x_0=m$$
- Continuous: $$\frac{dx}{dt}=b(t, x), x(0)=m$$

### Dynamic System

#### Stochastic
- Discrete: $$x_{j+1}=\Psi(x_j)+\xi_j,\; x_0=\xi_0$$
- Continuous: $$\frac{dX_t}{dt}=b(t,X_t)+\sigma\frac{dB_t}{dt},\; X_0=M$$

**Discrete Example**:
$$x_{j+1}=\pmatrix{0&1\\-1&0}x_j + \sigma z_j,\; x_0=\pmatrix{1\\1}$$

In [6]:
from dynamics import dsct, cnts

In [7]:
dsct.interact()

#### Notes:

- Red point is $x$ in previous math model.
- $x$ and $y$ in plot label simply means $x$- and $y$-coordinates.
- Click <button name="Observe">Observe</button> to plot data/observations (Represented by the horizontal red line). See slides in [__Observations/Data__](#Observations/Data).
- We can only observe the $y$-coordinate (the height) of our point. Mathematically, $h(x)=\bmatrix{0&1}x$.
- $\sigma$ controls randomness. Larger $\sigma$ results in larger perturbation. If set $\sigma\ne0$ and then set it back to 0, the perturbation "stays" in the system. (I.e., the red point does not go back to the original square, but follows a new square.)
- $\gamma$ controls noise in observations. Larger $\gamma$ results in larger difference between observation (red line) and $x_2$ ($y$-coordinate of red point)

**<a name='Continuous-Example:'/>Continuous Example**:
$$\frac{dX_t}{dt}=\pmatrix{0&1\\-1&0}X_t + \sigma\frac{dB_t}{dt},\; X_0=\pmatrix{0\\1}$$

In [8]:
cnts.interact()

#### More Notes:

Same as [here](#Notes:).

Read the following slides about observations first.

Now set both $\sigma$ and $\gamma$ nonzero. Imagine you can only see the horizontal red line (the observation). How can you guess where the red point is?

### Observations/Data
$$
\newcommand\F{\mathcal{F}}
\begin{align}
y_j&= h(x_j)+\eta_j\\
Y_t&= h(t, X_t)+\gamma\frac{dB_t}{dt}
\end{align}
$$ 

**Example:** 

$\newcommand\bmatrix[1]{\begin{bmatrix}#1\end{bmatrix}}$ $$h(x) = \bmatrix{0&1}x$$

Time to go back to previous examples and click `Observe`.

## Filtering
Given a dynamic system + observations:
$$
\begin{align}
x_{j+1}&=\Psi(x_j)+\xi_j & \frac{dX_t}{dt} &= b(t,X_t) + \sigma(t,X_t)\frac{dU_t}{dt}\\
y_j    &= h(x_j)+\eta_j  &             Y_t &= h(t,X_t) + \gamma(t,X_t)\frac{dV_t}{dt}
\end{align}
$$

Find information of $x$ at current time, given all the data before and at current time.

- Discrete:
$$P(x_j|Y_j),\; Y_j=\{y_0, y_1, \ldots, y_j\}$$
- Continuous:
$$E[X_t|\F_t]$$
where $\F_t$ is the $\sigma$-algebra generated by $\{Y_s, s\le t\}$

### Stochastic Settings

Let's consider the continuous case only:
$$
\begin{align}
\frac{dX_t}{dt} &= b(t,X_t) + \sigma(t, X_t)\frac{dU_t}{dt}\\
            Y_t &= h(t,X_t) + \gamma(t, X_t)\frac{dV_t}{dt}
\end{align}
$$
If we let $Z_t = \int_0^tY_sds$, then the above system can be rephrased as an SDE system:
$$
\begin{align}
dX_t &= b(t, X_t)dt + \sigma(t, X_t)dU_t\\
dZ_t &= h(t, X_t)dt + \gamma(t, X_t)dV_t
\end{align}
$$
Solve for $E[X_t|\F_t]$.

## Kalman Filter


### 1D Linear Filtering Problem

Consider the 1-dimensional linear filtering problem

$$
\newcommand\R{\mathbb{R}}
\begin{align}
dX_t&= F(t)X_tdt+C(t)dU_t;\;F(t), C(t)\in\R\\
dZ_t&= G(t)X_tdt+D(t)dV_t;\;G(t), D(t)\in\R
\end{align}
$$

Also we assume
$$X_0\sim N,\; Z_0=0,\; \lvert D(t)\rvert>\epsilon>0$$

Need to find: $E[X_t|\F_t]$

#### Definitions: 


- $\F_t:=\sigma(\{Z_s\mid 0\le s\le t\})$
- $\L=\L_t=\L(Z,t):=$ the closure in $L^2$ of all linear combinations:
$$c_0+c_1Z_{t_1}+c_2Z_{t_2}+\cdots+c_kZ_{t_k};\;0\le t_i\le t, c_j\in\R\;.$$

- $\K=\K_t=\K(Z,t):= \{\text{all $\F_t$ measurable RV's in $L^2$}\}$ 
- $\P_\L(X):=$ the projection of $X$ onto space $\L$.

where $L^2=L^2(\Omega, P)$

#### Fact 1: $E[X_t|\F_t]=\P_\K(X_t)=\P_\L(X_t)$

*Proof (Sketch):*

$\newcommand\hX{\hat{X}}
\newcommand\tX{\tilde{X}}$
Let $\hX=\P_\L(X)$, and let $\tX=X-\hX$. Then $\tX$ is orthogonal to every element in $\L$:

$$ E[\tX Z_s] = 0\;. $$

I.e., $\tX$ and $Z_s$ are uncorrelated. Since $X, Z_s$ are Gaussian process, $\tX$ is Gaussian. Then uncorrelated means independence. 

$\tX$ is independent of $\K$. Therefore $\tX=\P_\K(X)$

#### $\L(Z,t)$

Closure of all

$$
\begin{align}
 &c_0 + c_1Z_{t_1} + c_2Z_{t_2} + \cdots + c_kZ_{t_k}\\
=&c_0 + d_1\Delta Z_{t_1} + d_2\Delta Z_{t_2} + \cdots + d_k\Delta Z_{t_k}\\
\to&c_0 + \int_0^tf(s)dZ_s
\end{align}
$$

#### Idea
Basis ($\{d Z_{t_i}\}$) $\longrightarrow$ Orthogonal Basis $\{d N_{t_i}\}$ $\longrightarrow$ Orthonormal Basis $\{d R_{t_i}\}$

#### Find Orthogonal Basis $dN_t$

$$dN_t = dZ_t - \P_\L(dZ_t)\;, N_0=0\;.$$

Recall: $$dZ_t = G(t)X_tdt+D(t)dV_t,$$ and $dV_t$ is independent of $\K_t$.

Thus $$\P_\L(dZ_t)=G(t)\hX_tdt\;,$$
$$
\begin{align}
dN_t &= dZ_t-G(t)\hX_tdt\\
&=G(t)(X_t-\hX_t)dt+D(t)dV_t
\end{align}
$$

#### Properties of $N_t$

1. $E[N_t-N_s|\K_s]=0$ (Orthogonal increments.)
4. $\L(N,t)=\L(Z,t)$
2. $E[N_t^2]=\int_0^tD^2(s)ds$
3. $N_t$ is Gaussian

First two: $\{dN_t\}$ is an orthogonal basis.

1 + 3 + 4: $N_t$ is *almost* a Brownian.

#### Normalize $dN_t$

If we let

$$ dR_t = \frac{1}{D(t)}dN_t;\; t\ge0, R_0=0\;, $$

Then

1. $R_t$ is a Brownian motion;
2. $\L(N, t)=\L(R, t)$.

#### $E[X_t|\F_t]$

$$
\begin{align}
\hX_t &= E[X_t|\F_t] \\
&= \P_\K(X_t) = \P_{\L(R,t)}(X_t)\\
&= E[X_t] + \int_0^t\frac{\partial}{\partial s}E[X_tR_s]dR_s\;.
\end{align}
$$

### Kalman-Bucy Filter

**Theorem**: The solution $\hX_t=E[X_t|\F_t]$ of the 1D linear filtering problem satisfies the stochastic differential equation:

$$ d\hX_t=\left(F(t)-\frac{G^2(t)S(t)}{D^2(t)}\right)\hX_tdt+\frac{G(t)S(t)}{D^2(t)}dZ_t,\; \hX_0=E[X_0]$$

where $S(t)=E[(X_t-\hX_t)^2]=E[(E[\hX_t]-\hX_t)^2]$ is the variance of $\hX_t$ and satisfies the *Riccati equation*

$$\frac{dS}{dt}=2F(t)S(t)-\frac{G^2(t)}{D^2(t)}S^2(t)+C^2(t)\;.$$

#### Multi-Dimensional Kalman-Bucy Filter

Theorem: The solution $\hX_t=E[X_t|\F_t]$ of the 1D linear filtering problem satisfies the stochastic differential equation:

$$ d\hX_t=\left(F-SG^T(DD^T)^{-1}G\right)\hX_tdt+SG^T(DD^T)^{-1}dZ_t,\; \hX_0=E[X_0]$$

where $S(t)=E[(X_t-\hX_t)(X_t-\hX_t)^T]$ is the covariance matrix and satisfies the *Riccati equation*

$$\frac{dS}{dt}=FS+SF^T-SG^T(DD^T)^{-1}S+CC^T\;.$$

In [9]:
from dynamics import show_result, do_filter, plot_error
import numpy as np

In [10]:
do_filter(init_guess=[50,50],S0=np.eye(2)*50, ic=[0, -1], sigma=.1, gamma=.1, dt=0.05)

About this function call:

- `init_guess`: initial guess.
- `ic`: true initial condition.
- `sigma`: $\sigma$ in [continuous example](#Continuous-Example:).
- `gamma`: $\gamma$ in [observations](#Observations/Data)
- `dt`: length of timeinterval for simulation.

Feel free to modify inputs and rerun the above function. You also need to rerun the following functions to see the result.

Remember to adjust the `lim` parameter in `show_result()` if you want to play with different parameters.

In [11]:
show_result(lim=5) # Change lim for different plot limits.

Green dot is the guess of where red point is.

If you haven't modified the above function yet, the initial guess is `[50,50]` and is outside of the plot. But after only a few timesteps, the guess will get pretty close to true state (red dot).

In [12]:
plot_error()

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='error', orientation='vertical', scale=LinearSca…

If everything works as expected, you'll see an error that starts large (if initial guess is bad), but shrinks fast, and stays small.

Impressive?